# 1 - Perkenalan

Name : Jason Rich Darmawan Onggo Putra

Batch : 016 RMT

Data set Overview : 

Objective :

# 2 - Import Libraries

In [1]:
# Data Loading
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np

# 3 - Data Loading

- columns that are not good separator of the target: ['customerID']

In [2]:
# Load the data set
df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Preview the data set
with pd.option_context("display.max.columns", None):
    display(df.head())

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


> [Nominal data (from the Latin word “nomen” meaning “named” data), is data that names or labels variables without a numerical value. The simplest example would be “yes” or “no.” These are two categories, but there is no way to order them from highest to lowest or best to worst.](https://www.google.com/search?q=is+yes+and+no+ordinal&oq=is+yes+and+no+ordinal&aqs=edge..69i57j69i64.3078j0j1&sourceid=chrome&ie=UTF-8#:~:text=7%20Considerations%20for,com%20%E2%80%BA%20Blog)

- Some nominal columns have somewhat ordered category e.g column 'OnlineSecurity' have values 'Yes', 'No', 'No internet service'. However, the ordered category argument is weak, e.g when we ask Is 'No' better than 'No internet service'?. Therefore, we concludes it as nominal columns.

- nominal columns: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod']
- numerical columns: ['tenure', 'MonthlyCharges', 'TotalCharges']
- ordinal columns: ['Contract']

- target is nominal: 'Churn'

TODO
- [ ] Change nominal columns dtypes to 'category' with unordered categories.
  - double check 'SeniorCitizen'.
- [ ] Change 'Contract' dtypes to 'category' with ordered categories.
- [ ] Change 'TotalCharges' dtypess to 'float64' (dedicated for numerical columns).
- [ ] Change 'Churn' dtypes to 'category' with ordered categories.

In [3]:
# Check dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
# TotalCharges have ' ' value.
# df['TotalCharges'].astype('float64')

In [4]:
# Define columns
# Philosophy: Do not change anything until Data Preprocessing to simulate the real world.
nom_cols = [
    'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
    'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod'
]
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
ord_cols = ['Contract']
target = 'Churn'

cat_cols = np.concatenate([nom_cols, ord_cols], axis=0)

# 4 - Exploratory Data Analysis (EDA)

In [5]:
# Central Tendency categorical columns
df[cat_cols].astype('category').describe()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,PaymentMethod,Contract
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,2,2,2,2,2,3,3,3,3,3,3,3,3,2,4,3
top,Male,0,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Yes,Electronic check,Month-to-month
freq,3555,5901,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,4171,2365,3875


# 5 - Data Preprocessing

In [ ]:
# Change nominal features dtypes to 'object'.
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')

In [ ]:
# Change ordinal features dtypes to 'category'
Contract_cat = CategoricalDtype(categories=['Month-to-month', 'One year', 'Two year'])
df['Contract'] = df['Contract'].astype(Contract_cat)

In [ ]:
# Change numerical features dtypes to number
# errors='coerce' invalid parsing will be set as NaN.
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [ ]:
# Check missing values
df.isnull().sum()

# 6 - Model Definition

# 7 - Model Training

# 8 - Model Evaluation

# 9 - Model Saving

# 10 - Model Inference

# 11 - Pengambilan Keputusan